In [324]:
# https://en.bitcoin.it/wiki/Protocol_documentation

# from scapy import *
from scapy.all import *
import cryptos as crypts
import pprint as pp
import json as js

In [485]:
#p=sniff(count=10000, filter="tcp port 8333")
#p=sniff(count=1000, filter="tcp port 8333 and dst host 178.162.214.41")
p=sniff(count   = 100, 
        filter  = "tcp port 8333 and dst host 178.162.214.41", 
        lfilter = lambda x: ((Raw in x) and x[Raw].load.hex().find('f9beb4d97478') > -1))
        #x[IP].frag > 0 or x[IP].flags == 1)

In [ ]:
# p2 = [packet for packet in p if len(packet)>300]
# len(p2)
ip_list = []
for packet in p:
    ip_list.append(packet[IP].src)

lot = []
for ip in set(ip_list):
    lot.append((len(p.filter(lambda x: x[IP].src==ip)),ip))
    
sorted(lot, key=lambda freq: freq[0], reverse=True)

## تلاش خودم برای تشخیص محتوای تراکنش خام

In [572]:
# https://en.bitcoin.it/wiki/Protocol_documentation#tx
# https://www.savvius.com/networking-glossary/tcp_ip_overview/ip/ip_fragmentation/
def get_flags (packet):
    if (packet[IP].flags == 2) :
        print ("Don't Fragment")
    elif (packet[IP].flags == 0):
        print ("More Fragment")
    elif (packet[IP].flags == 1):
        print ("Last Fragment")
    return

def get_command (packet):
    return (packet[Raw].load[4:16]).decode('ascii').replace('\x00',"")

def get_len (packet):
    return int(struct.unpack("4s",(packet[Raw].load)[16:20][::-1])[0].hex(),16)

def get_verwit(packet):
    version = packet[Raw].load[24:28].hex()
    witness = '0001' if(packet[Raw].load[28:30].hex()=='0001') else 'No Witness'
    return (version,witness)

def get_insnum (packet):
    input_num = int(packet[Raw].load[30:31].hex(),16)
    return input_num

def get_ins (packet):
    input_num = get_insnum(packet)
    print ('Number of Inputs = ',input_num)
    offset = 0;
    while (input_num > 0):
        print('Input Num.',input_num)
        prev_out   = packet[Raw].load[offset+31:offset+63].hex()
        prev_index = packet[Raw].load[offset+63:offset+67].hex()
        print ('Previous Output = ',prev_out)
        print ('Previous Output index = ',prev_index)
        script_len = int(packet[Raw].load[offset+67:offset+68].hex(),16)
        script     = packet[Raw].load[offset+68:offset+68+script_len].hex()
        print ('Script = ',script)
        if  (packet[Raw].load[offset+68+script_len:offset+68+script_len+4].hex() != 'ffffffff'):
            print ('Something is Wrong!')
            return
        input_num -= 1
        offset += 41 + script_len
    return offset

def get_outs (packet,offset):
    output_num = int(packet[Raw].load[offset:offset+1].hex(),16)
    print ('Number of outputs = ',output_num)
    while (input_num > 0):
        print('Input Num.',input_num)
        prev_out   = packet[Raw].load[offset+31:offset+63].hex()
        prev_index = packet[Raw].load[offset+63:offset+67].hex()
        print ('Previous Output = ',prev_out)
        print ('Previous Output index = ',prev_index)
        script_len = int(packet[Raw].load[offset+67:offset+68].hex(),16)
        script     = packet[Raw].load[offset+68:offset+68+script_len].hex()
        print ('Script = ',script)
        if  (packet[Raw].load[offset+68+script_len:offset+68+script_len+4].hex() != 'ffffffff'):
            print ('Something is Wrong!')
            return
        input_num -= 1
        offset += 41 + script_len
    return offset

## یک بسته اترنت را گرفته و دیکد می کند

In [555]:
def DecodeTx (packet):
    if (len(packet[Raw].load) > 24):
        if(packet[Raw].load.hex()[0:12] == 'f9beb4d97478'):
            return crypts.deserialize(packet[Raw].load.hex()[48:])
        elif(packet[Raw].load.hex()[:8] in ['01000000','02000000']):
            return crypts.deserialize(packet[Raw].load.hex())
        else:
            return 0
    else:
        return  0

In [487]:
DecodeTx(p[10])

{'ins': [{'outpoint': {'hash': '46656830db07301add52b32532fe974776bf616b50b30061bd3d8431bf1744fb',
    'index': 0},
   'script': '4730440220128a1214dcaca53f3944214635265212ed90c5bfcb195b0b64900dec19cb1c7f02203fc7e6a6075973e27081af0eebeec0c4e1c1289ac2a3e568918d42bf90a83b0e012103ed90364e7ca14b6cb4dabdba5fea0babdc91c6b1148c04a0e9ddc86518fb391f',
   'sequence': 4294967295}],
 'outs': [{'value': 1230876,
   'script': '76a914330e36debb968993f2f758f520538730539f105588ac'}],
 'version': 1,
 'locktime': 0}

## محاسبه مشخصه تراکنش از روی مقدار خام آن

In [302]:
def GetTxID (packet):
    import hashlib,binascii
    hash = hashlib.sha256(hashlib.sha256(binascii.unhexlify(packet[Raw].load.hex()[48:])).digest()).digest()
    print ("TXID: ", hash[::-1].hex())

In [486]:
GetTxID(p[10])

TXID:  dccb0dbaf2366b38610ae5bce956057af52d437fd9bcbcab7e006f36ed11ce56


## محاسبه مقدار هگز یک آدرس از روی مقدار مبنای ۵۸ متداول آن

In [516]:
import base58
base58.b58decode('1EyvRCg3jUDQhRngvBYEcd5fkEtUadFrFF').hex()

'00995d85003f48a4d5f4a51462adf27b6f750f57f57d11f86e'

## یک بسته اترنت را گرفته و آدرس کیف پول هایی که به عنوان ورودی تراکنش شرکت دارند را بر می گرداند

In [554]:
# مشکل آدرس های مربوط به segwit
# https://github.com/bitcoin/bips/blob/master/bip-0142.mediawiki

def ResolveOutAddr(packet,verbos):
    import requests
    main_url = 'http://127.0.0.1:8332/rest/'
    decpack = DecodeTx(packet)
    addr_val = {}
    if (decpack):
        tx = js.loads(js.dumps(decpack))
        addr_val = {}
        index = tx['ins'][0]['outpoint']['index']
        txhash = tx['ins'][0]['outpoint']['hash']
        url = main_url + "tx/" + txhash +'.json'
        tx_response = requests.get(url)
        result = js.loads(tx_response.content)
        itm = result['vout']
        if (itm[0]['n'] in [index]):
            if(itm[0]["scriptPubKey"]['type'] in ['scripthash','pubkeyhash']):
                addr_val.update({itm[0]["scriptPubKey"]['addresses'][0] : itm[0]["value"]})
            elif(verbos):
                print(itm[0]["scriptPubKey"]['type'] + '  Detected') 
    elif(verbos):
        print("packet too small")
    return addr_val

In [563]:
for i in range(0,len(p)):
    res = ResolveOutAddr(p[i],0)
    if (res):
        print(i,": ",res,"IP = ",p[i][IP].src)

4 :  {'1PzX6fCijoT8rBzzN3kZoqi7YGhTBeBEH': 2.86237975} IP =  54.90.249.112
10 :  {'1EyvRCg3jUDQhRngvBYEcd5fkEtUadFrFF': 0.01231424} IP =  94.130.69.46
14 :  {'1EMq5cph7qMf25vTQateKoKXu7t3VxUGxg': 2.8984958} IP =  94.130.69.46
26 :  {'1LfV9MMw64iKtFkVHKV1RXFXNRSyCQffN9': 2.71111006} IP =  188.209.49.20
28 :  {'1AmkxTZrggirm7FrgqN65YFLTEFw1tWg1i': 0.00388492} IP =  81.7.10.238
30 :  {'1MbT49WAQAtQ3Df7Ei1qjfdRnzLRZuWJA3': 0.01674678} IP =  46.227.176.197
35 :  {'12FpK4dgjkyb3HCP86GgAUXDe3EFh4MXgL': 2.66045046} IP =  75.98.155.228
68 :  {'15tD4B6CPdqBFyR6CTnwriob9u1wbGnjww': 2.65704314} IP =  151.80.36.205
86 :  {'1Aezvp3ngBjMSukiDjumkpyZBpsbEXdB2T': 2.8984963} IP =  46.227.176.197
97 :  {'1NHQJVSCzAhwAC3g7KrgGVLXjv2ktNs3BN': 2.91888562} IP =  178.33.24.77
